# Keeletehnoloogia 3. kodutöö - autori ennustamine
Programmile antakse kolme autori ingliskeelsed korpused, mis on lihtsad tekstifailid, kuhu on ilma igasuguse märgenduseta järjest kirjutatud autori eri teosed (ehk siis eeltöötluse tuleb ise teha). Mudel peab antud uue (korpusevälise) lause põhjal ennustama, kes oli selle lause autor.  
Kirjuta valmis funktsioonid *train* ja *predict* nii, et kaasasolev testplokk korrektselt töötaks. Muid plokke ja funktsioone võib lisada ja kustutada, kuid väljatrükid ja suured arvutused hoitagu testploki sees.  
  
**NB!** Ärge kirjutage sisse täisteid, kommenteerige oma koodi, jooksutage enne esitamist nullist läbi ja ärge unustage ülesande esitamisel *Moodle'i* tekstikasti oma mudeli inimloetavat kirjeldust panna. Edu!


**Impordid** - lisage, mis hing ihaldab, kuni neid eraldi installima ei pea. EstNLTK ja muu 0. praktikumis läbi käinu võib muidugi olla.

In [1]:
# Sisuliselt kõik samad asjad mis prakitkumis kasutati
from time import time
# Regex failide mugavamaks sisselugemiseks
import re
import nltk
import random
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.models import load_model
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Embedding

## Treenimine

In [2]:
# Funktsioon andmete treening-, test-, ja valideerimishulgaks jagamiseks
# Lõppversioonis 90% andmetest treenimiseks, ja 10% võrdselt jagatud test- ja valideerimishulga vahel
# Kokkuvõttes siis: 90% andmetest treenimishulk, 5% valideerimishulk ja 5% testhulk
def distribute(length):
    train = int(round(length*0.9))
    test = train + int(round((length-train)*0.5))
    # Tagastan ainult treenimis- ja testhulga alguste indeksid, kuna
    # pythoni listi tükeldamise abil saame valideerimishulga kätte
    # võttes nendevahelised andmed
    return train, test

In [3]:
def preprocess(data, target, tok=None):
    # Loome tokenizeri, kui seda veel pole, ning
    # kasutame seda ka test- ja valideerimishulgal
    # sisuliselt täpselt sama mis praktikumi kolmandas osas tehtud sai
    if tok is None:
        tok = Tokenizer(char_level=False)
        tok.fit_on_texts(data)
        
    sent_seqs = tok.texts_to_sequences(data)
    X = sequence.pad_sequences(sent_seqs, maxlen=30)
    return X, to_categorical(target, num_classes=3), tok
    

In [4]:
def train(fail1, fail2, fail3):
    # Sisendiks: kolme faili nimed
    # Kõigepealt loon 2 listi, texts ja labels, texts listi lisan järjest kõik korpused
    # lausestatud kujul nii, et kõigepealt eemaldan reavahetused ja seejärel mitmekordsed tühikud
    # labels listi panen vastava korpuse autori indeksi
    texts = list()
    labels = list()
    failid = [fail1, fail2, fail3]
    for i in range(len(failid)):
        with open(failid[i], 'r', encoding='utf-8') as f:
            texts.append(nltk.sent_tokenize(re.sub('\s+', ' ', f.read().replace('\n', ' '))))
            labels.append(i)
    
    # Tasakaalustan andmed leides kõige väiksema korpuse suuruse
    # Idee seisneb pikemate korpuste lõppude ära lõikamises, et oleks aus
    # segan eelnevalt pikema korpuse laused suvaliselt
    # siin ei kaota me autorite indekseid, sest me ei sega texts listi kuidagi
    # vaid texts listi sees olevaid lausestatud korpusi
    unbiased = list()
    shortest = 10000000
    for elem in texts:
        if len(elem) < shortest:
            shortest = len(elem)
    for elem in texts:
        random.shuffle(elem)
        if len(elem) >= shortest:
            unbiased.append(elem[:shortest])
    
    # Seon korpused neile vastava autori indeksiga
    labelled_texts = zip(unbiased, labels)
    
    # Loon 2 listi, sentences listi panen
    # järjest lauseid ja y listi vastavale indeksile
    # vastava korpuse autori indeksi
    sentences = []
    y = []
    for item, cls in labelled_texts:
        sentences.extend(item)
        y.extend([cls] * len(item))
    
    # Segan mõlemad listid ilma lause -> autor suhet rikkumata
    c = list(zip(sentences, y))
    random.shuffle(c)
    sentences, y = zip(*c)
    
    # Leian distribute funktsiooni abil vastavalt andmestiku suurusele
    # indeksit jaotamaks need treenimis-, test- ja valideerimishulkadeks
    train, test = distribute(len(sentences))
    X_train = sentences[:train]
    y_train = y[:train]
    X_val = sentences[train:test]
    y_val = y[train:test]
    X_test = sentences[test:]
    y_test = y[test:]
    
    # Siin viin saadud hulgad vastavalt praktikumis nähtule
    # närvivõrgule seeditavale kujule
    X_train_proc, y_train_proc, tok = preprocess(X_train, y_train)
    X_val_proc, y_val_proc,  _  = preprocess(X_val, y_val, tok)
    X_test_proc, y_test_proc,  _  = preprocess(X_test, y_test, tok)
    
    # Närvivõrgule ette antava sõnastiku suurus, koos nulliga +1
    vocabulary_size = len(tok.word_index) + 1
    
    # Loon mudeli mis võtab sisendiks järjendi
    model = Sequential()
    # Embedding kihis määran lause pikkuseks sõnastiku suuruseks enne arvutatud
    # suuruse, leidsin kõige parema täpsuse kolme peidetud LSTM kihiga
    # Ülejäänud mudeli konstruktsioon sarnaselt praktikumile,
    # lisaks otsustasin kasutada täpselt 4 epohhi, kuna valideerimisandmestikuga
    # hakkas pärast neljandat epohhi täpsus vähenema, ning treenimishulgaga
    # saadi juba ligi 95% niikuinii kätte
    model.add(Embedding(vocabulary_size, 30, input_length=30))
    model.add(LSTM(60, return_sequences=True))
    model.add(LSTM(60, return_sequences=True))
    model.add(LSTM(60, return_sequences=False))
    model.add(Dense(len(failid), activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()
    model.fit(X_train_proc, y_train_proc, epochs=4, batch_size=64, validation_data=(X_val_proc, y_val_proc))
    tulemus = model.evaluate(X_test_proc, y_test_proc)
    print(tulemus)
    print("Testhulgal oli täpsus {0}%, kaofunktsiooni väärtus {1}.".format(100*tulemus[1],round(tulemus[0],6)))
    
    # Salvestan mudeli mugavamaks taaskasutamiseks
    model.save("lauseKlassifitseerija.h5")
    # Tagasta mudel (nt närvivõrk), mis suudab ennustada, millisesse kolmest failist võiks lause kuuluda.
    # Tagastan mudeli ja tokenizeri ennustamiseks
    return (model, tok)

## Ennustamine



In [5]:
def predict(mudel, lause):
    # Sisendiks: 
    # mudel - meetodi 'train' tagastatud mudel
    # lause - klassifitseerimist vajav string, sõnestamata puhtal kujul
    # viin lause närvivõrgule seeditavale kujule
    lause = mudel[1].texts_to_sequences([lause])
    # Ennustan autori indeksi algsest autorite nimekirjast
    ennustus = mudel[0].predict_classes(lause)
    # Tagasta klass - täisarvuline väärtus hulgast {1,2,3}
    # Kuna tagastama peab väärtuse hulgast {1, 2, 3} ja minu mudel
    # tagastab vastava indeksi autorite listist, mille pikkus on 3,
    # siis õige klassi saamiseks liidan ennustatud väärtusele 1.
    return ennustus[0]+1

## Testimine

In [6]:
if __name__ == "__main__":
  # Igasugu väljatrükkidega asjad võivad olla siin.

  # Treeni nende kolme faili alusel
  algusaeg = time()
  m = train("korpused/janeausten.txt", "korpused/shakespeare.txt", "korpused/poe.txt")
  aega = time()-algusaeg
  print("Treenimiseks kulus {} minutit ja {} sekundit".format(round(aega/60),round(aega%60,2)))
  # Meeldetuletuseks: võiks jääda 15 minuti piiridesse

  # Ennusta, mis autori teos võiks see lause olla.
  algusaeg = time()
  vastus = predict(m, "To be or not to be, that is the question.") 
  aega = time()-algusaeg
  print("Ennustamiseks kulus {} sekundit".format(round(aega,2)))
  # Meeldetuletuseks: võiks jääda 10 sekundi piiridesse
  # Loodetavasti on vastuseks 2 - "shakespeare.txt"
  print("Vastus: ",vastus)


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 30, 30)            968370    
_________________________________________________________________
lstm (LSTM)                  (None, 30, 60)            21840     
_________________________________________________________________
lstm_1 (LSTM)                (None, 30, 60)            29040     
_________________________________________________________________
lstm_2 (LSTM)                (None, 60)                29040     
_________________________________________________________________
dense (Dense)                (None, 3)                 183       
Total params: 1,048,473
Trainable params: 1,048,473
Non-trainable params: 0
_________________________________________________________________
Epoch 1/4
720/720 [==============================] - 41s 57ms/step - loss: 0.4789 - accuracy: 0.7952 - val_loss:

## Mudeli kirjeldus

Lähenesin sarnaselt praktikumis soovitatule. Failide sisu vaadates avastasin, et osades on palju tühje ridu, mis sisaldavad ainult reavahetust '\n' ja mõnes kohas ka liigseid tühikuid, niiet kohe kogu sisu lausestades ei saaks ilmselt eriti häid lauseid. Sellest tulenevalt otsustasin kõigepealt reavahetused asendada tühikutega ning seejärel mitmekordsed tühikud ühe tühikuga. Seejärel tasakaalustasin klassid, lausestasin ja klassifitseerisin lausehaaval. Klassifitseeritud lausetest panin kokku kolm andmehulka: treening-, test- ja valideerimishulk. Mis lõpptulemuses on vastavalt 90, 5 ja 5 %-lise osakaaluga. Otsustasin enda tehtud andmestikujaotuse lõppversiooni sisse jätta, kuna see 10% niikuinii eriti ei tõstaks mu täpsust ja niiviisi saab tunduvalt parema ülevaate mudeli töötamisest. Närvivõrgu parameetreid katsetades jõudsin järelduseni, et kuna antud andmestikus on nii väga pikki kui ka väga lühikesi lauseid, siis ei ole otseselt mõistlik võtta vektori pikkuseks kõige pikema lause pikkust. Keskmise lause pikkuseks ennustasin umbes 30 sõna, millega ka kõige parema tulemuse sain. LSTM kihte lisasin 3 vastavalt praktikumis näidatule, neuronite arvuga mängisin katse-eksituse meetodil ja jätsin need millega parima tulemuse leidsin. Lõpuks salvestasin mudeli failina 'lauseKlassifitseerija.h5'.

Joosep Tavits 30/03/2021 01:49

## Tagasiside

Leian, et hetkeseisuga minu jaoks on kõige mugavam, kui praktikumides ei eeldata ülesannete kaasa tegemist, kuna mitme ekraani puudumisel on zoomi akent ja lokaalset jupyter notebooki kõrvuti suhteliselt võimatu hallata, kui samal ajal peab keerulistest asjadest ka veel aru saama. Isiklikult on kodutööd aidanud kõike mida praktikumis kuulnud olen, rakendada, ning millestki puudust ei tunne. Lisaks veel mainiks, et täpsuse peale tegemise kodutööd on väga põnevad! :)